The implementation of the selective greedy method to the polyhedral sets, as given on the page 38.

In [ ]:
#Block 1: loading packages
import numpy as np
from scipy import linalg as la
from numpy.linalg import norm
from pulp import *
import time

In [ ]:
#Block 2: setting the rounding and precision parameters
prec = 8 #rounding parameter
Eps = 10**(-prec)

In [ ]:
#Block 3: functiong for creating a random normalized vector
def normvec():
    ranvec = lambda: np.random.rand(dim,)
    b0 = ranvec()
    b0 = b0/float(norm(b0))
    b0 = np.round(b0,prec)
    
    return b0

In [ ]:
#Block 4: LP solver, determening the solution which maximizes the objective function
def lpsolve(v,b):
    prob = LpProblem('lp',LpMaximize)
    
   
    x = [LpVariable('x%d' %i,0,1) for i in xrange(dim)] #unknown variables, bounded by (0,1)
    
    
    obj = 0
    const = [0]*N 
    for i in xrange(dim):
        obj += x[i]*v[i] #the objective function
        const = [const[j] + x[i]*b[j][i] for j in xrange(N)] #the constraints
        
  
    prob += obj
    
    #the constraints
    for j in xrange(N):
        prob += const[j] <= 1

    
    prob.solve()
    
    sol = [x[i].varValue for i in xrange(dim)] #optimal solution
    sol = np.round(sol,prec)
    
    return sol

In [ ]:
#Block 5: selective power method 
def pwrmthd(A):
    A = A + np.identity(dim) #see the remark on page 28
    v0 = np.array([1 for i in xrange(dim)]) #starting vector of all ones 
    v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    v1 = np.round(v1,prec)
    while norm(v0-v1) > Eps*10: #the precision parametar $\varepsilon$ 
        v0 = v1
        v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    return np.round(v1,prec-1)

In [ ]:
#Block 6: function for computing the spectral radius
def leading(A):
    
    evals = np.linalg.eig(A)[0] #set of eigenvalues 
    return np.amax(np.real(evals)) #spectral radius

In [ ]:
#Block 7: selective greedy method for polyhedral sets
def greedy_polyMAX(A): 
    
    Z = np.copy(A)
    Z0 = np.copy(Z)
    
    start = time.clock() #time counter
    count = 0 #the iteration counter
    
    while True:
        v = pwrmthd(Z) 
        count += 1
        
        for i in xrange(dim): #solving the LP problems
            a = lpsolve(v,F[i])
            Z0[i] = a
       
    
        if (Z0 == Z).all(): #checking wether the new matrix is truly optimal
            run = np.round(time.clock() - start,2)
            spect = np.round(leading(Z),2)
            return spect, count, run
        
        else: 
            Z = np.copy(Z0)

In [ ]:
#Block 8: running the code
dim = 50 #seting the dimension
N = 20 #seting the number of constraints; the actuall number of constraints will be 2d + N
F = [[normvec() for i in xrange(N)] for x in xrange(dim)] #generating the polyhedral product family
A = np.array([normvec() for x in xrange(dim)]) #generating the starting matrix
G = greedy_polyMAX(A) #running the selective greedy method on F
print G